# Monte Carlo RSM Simulator

This notebook demonstrates how to use the Monte Carlo simulator to analyze RSM (Replicated State Machine) deployments. We'll explore:

1. Setting up node configurations with failure distributions
2. Running single simulations
3. Running Monte Carlo simulations for statistical analysis
4. Visualizing availability and time-to-data-loss distributions

In [ ]:
# Imports
from powder.simulation import (
    # Time units
    Seconds, hours, days, minutes,
    # Distributions
    Exponential, Weibull, Normal, Uniform, Constant,
    # Node
    NodeConfig, NodeState,
    # Network
    RegionPair, NetworkConfig, NetworkState,
    # Cluster
    ClusterState,
    # Strategy
    NoOpStrategy, SimpleReplacementStrategy,
    # Simulator
    Simulator,
)
from powder.monte_carlo import run_monte_carlo, MonteCarloConfig, MonteCarloRunner
from powder.graphing_utils import (
    make_pdf_histogram,
    make_cdf_plot,
    make_availability_boxplot,
    make_time_to_loss_comparison,
)
import numpy as np

## 1. Define Node Configuration

Each node has:
- **region**: Geographic location
- **cost_per_hour**: Dollar cost to run the node
- **failure_dist**: Distribution for time until transient failure
- **recovery_dist**: Distribution for time to recover from failure
- **data_loss_dist**: Distribution for time until permanent data loss (e.g., disk failure)
- **sync_rate_dist**: Distribution for sync rate (seconds of data per second of real time)
- **spawn_dist**: Distribution for time to spawn a new replacement node

In [ ]:
# Create a realistic node configuration
def make_standard_node_config(region: str = "us-east") -> NodeConfig:
    return NodeConfig(
        region=region,
        cost_per_hour=0.50,  # $0.50/hour per node
        # Transient failures: ~1 per week on average
        failure_dist=Exponential(rate=1 / days(7)),
        # Recovery time: 5-15 minutes
        recovery_dist=Uniform(low=minutes(5), high=minutes(15)),
        # Permanent data loss: ~1 per year (disk failure rate)
        data_loss_dist=Exponential(rate=1 / days(365)),
        # Sync rate: node can download 3 seconds of data per 1 second of real time
        sync_rate_dist=Constant(3.0),
        # Spawn time: 10-20 minutes to provision a new node
        spawn_dist=Uniform(low=minutes(10), high=minutes(20)),
    )

node_config = make_standard_node_config()
print(f"Node config: {node_config.region}, ${node_config.cost_per_hour}/hr")

## 2. Create Initial Cluster

A cluster consists of multiple nodes. For a 3-node cluster, we need a quorum of 2 nodes to commit new requests.

In [ ]:
def create_cluster(num_nodes: int = 3, regions: list[str] | None = None) -> ClusterState:
    """Create a fresh cluster with the specified number of nodes."""
    if regions is None:
        regions = ["us-east", "us-west", "eu-west"]
    
    nodes = {}
    for i in range(num_nodes):
        region = regions[i % len(regions)]
        config = make_standard_node_config(region)
        nodes[f"node{i}"] = NodeState(
            node_id=f"node{i}",
            config=config,
            last_up_to_date_time=Seconds(0),
        )
    
    return ClusterState(
        nodes=nodes,
        network=NetworkState(),
        target_cluster_size=num_nodes,
    )

cluster = create_cluster(3)
print(f"Created cluster: {cluster}")
print(f"Quorum size: {cluster.quorum_size()}")
print(f"Can commit: {cluster.can_commit()}")

## 3. Run a Single Simulation

Let's run a simulation for 30 days and observe the behavior.

In [ ]:
# Run simulation for 30 days with no intervention strategy
cluster = create_cluster(3)
simulator = Simulator(
    initial_cluster=cluster,
    strategy=NoOpStrategy(),
    seed=42,
    log_events=True,  # Keep event log for debugging
)

result = simulator.run_for(days(30))

print(f"Simulation ended: {result.end_reason}")
print(f"End time: {result.end_time / 86400:.2f} days")
print(f"Availability: {result.metrics.availability_fraction() * 100:.2f}%")
print(f"Total cost: ${result.metrics.total_cost:.2f}")
print(f"Events processed: {len(result.event_log)}")

In [ ]:
# Show first 10 events
print("First 10 events:")
for event in result.event_log[:10]:
    print(f"  t={event.time/3600:.2f}h: {event.event_type.value} on {event.target_id}")

## 4. Run Monte Carlo Simulations

To get statistical properties, we run many simulations and aggregate the results.

In [ ]:
# Run 100 simulations, each for 1 year, with NoOp strategy
results_no_op = run_monte_carlo(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=NoOpStrategy,
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

print(results_no_op.summary())

In [ ]:
# Now run with SimpleReplacementStrategy that spawns replacements for failed nodes
def make_replacement_strategy():
    return SimpleReplacementStrategy(
        default_node_config=make_standard_node_config(),
        scale_down_threshold=3,  # Only scale down if 3+ nodes fail simultaneously
    )

results_with_replacement = run_monte_carlo(
    cluster_factory=lambda: create_cluster(3),
    strategy_factory=make_replacement_strategy,
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

print(results_with_replacement.summary())

## 5. Visualize Results

### 5.1 Availability Distribution

In [ ]:
# PDF histogram of availability
avail_samples = [a * 100 for a in results_with_replacement.availability_samples]
fig = make_pdf_histogram(
    samples=avail_samples,
    title="Availability Distribution (with Replacement Strategy)",
    x_label="Availability (%)",
    bins=30,
)
fig.show()

In [ ]:
# Compare availability between strategies
fig = make_availability_boxplot(
    results_list=[results_no_op, results_with_replacement],
    labels=["No Intervention", "With Replacement"],
    title="Availability Comparison: No Intervention vs Replacement Strategy",
)
fig.show()

### 5.2 Time to Data Loss Distribution

In [ ]:
# CDF of time to actual data loss (NoOp strategy)
loss_times_days = [t / 86400 for t in results_no_op.time_to_actual_loss_samples_filtered()]

if loss_times_days:
    fig = make_cdf_plot(
        samples=loss_times_days,
        title="Time to Data Loss CDF (No Intervention)",
        x_label="Days",
        show_percentiles=[50, 90, 99],
    )
    fig.show()
else:
    print("No data loss events occurred in the simulations")

In [ ]:
# PDF histogram of time to data loss
if loss_times_days:
    fig = make_pdf_histogram(
        samples=loss_times_days,
        title="Time to Data Loss PDF (No Intervention)",
        x_label="Days",
        bins=30,
    )
    fig.show()

## 6. Parameter Sensitivity Analysis

Let's see how the number of nodes affects availability and time to data loss.

In [ ]:
# Compare 3, 5, and 7 node clusters
results_by_size = {}

for num_nodes in [3, 5, 7]:
    results = run_monte_carlo(
        cluster_factory=lambda n=num_nodes: create_cluster(n),
        strategy_factory=make_replacement_strategy,
        num_simulations=50,
        max_time=days(365),
        stop_on_data_loss=True,
        seed=42,
    )
    results_by_size[num_nodes] = results
    print(f"\n{num_nodes}-node cluster:")
    print(results.summary())

In [ ]:
# Compare availability across cluster sizes
fig = make_availability_boxplot(
    results_list=list(results_by_size.values()),
    labels=[f"{n} nodes" for n in results_by_size.keys()],
    title="Availability by Cluster Size",
)
fig.show()

In [ ]:
# Compare time to data loss across cluster sizes
fig = make_time_to_loss_comparison(
    results_list=list(results_by_size.values()),
    labels=[f"{n} nodes" for n in results_by_size.keys()],
    title="Time to Data Loss by Cluster Size",
    actual=True,
    time_unit="days",
)
fig.show()

## 7. Custom Failure Scenarios

Let's simulate a scenario with unreliable nodes (higher failure rates).

In [ ]:
def make_unreliable_node_config(region: str = "us-east") -> NodeConfig:
    """Node with higher failure rates."""
    return NodeConfig(
        region=region,
        cost_per_hour=0.30,  # Cheaper but less reliable
        # Transient failures: ~1 per day
        failure_dist=Exponential(rate=1 / days(1)),
        # Recovery time: 10-30 minutes (slower recovery)
        recovery_dist=Uniform(low=minutes(10), high=minutes(30)),
        # Permanent data loss: ~1 per 6 months
        data_loss_dist=Exponential(rate=1 / days(180)),
        sync_rate_dist=Constant(2.0),  # Slower sync
        spawn_dist=Uniform(low=minutes(15), high=minutes(30)),
    )

def create_unreliable_cluster(num_nodes: int = 5) -> ClusterState:
    nodes = {}
    for i in range(num_nodes):
        config = make_unreliable_node_config()
        nodes[f"node{i}"] = NodeState(
            node_id=f"node{i}",
            config=config,
        )
    return ClusterState(
        nodes=nodes,
        network=NetworkState(),
        target_cluster_size=num_nodes,
    )

# Run simulation with unreliable nodes
results_unreliable = run_monte_carlo(
    cluster_factory=lambda: create_unreliable_cluster(5),
    strategy_factory=lambda: SimpleReplacementStrategy(
        default_node_config=make_unreliable_node_config(),
    ),
    num_simulations=100,
    max_time=days(365),
    stop_on_data_loss=True,
    seed=42,
)

print("Unreliable 5-node cluster:")
print(results_unreliable.summary())

In [ ]:
# Compare reliable vs unreliable clusters
fig = make_availability_boxplot(
    results_list=[results_by_size[5], results_unreliable],
    labels=["Reliable 5-node", "Unreliable 5-node"],
    title="Reliable vs Unreliable Nodes",
)
fig.show()

## Summary

This simulator allows you to:

1. **Model RSM deployments** with realistic node configurations
2. **Define failure distributions** (exponential, Weibull, uniform, etc.)
3. **Implement custom strategies** for handling failures
4. **Run Monte Carlo simulations** to get statistical results
5. **Visualize results** with PDFs, CDFs, and comparison plots

Key metrics computed:
- **Availability**: Fraction of time the system can accept commits
- **Time to potential data loss**: When quorum is first lost
- **Time to actual data loss**: When all up-to-date nodes fail
- **Total cost**: Accumulated infrastructure costs